In [10]:
import pandas as pd
import datetime
df = pd.read_csv('csv/game_summary.csv')
print('ESTE ES EL SHAPE:')
print(df.shape)
print('INFO DE DF')
df.info

ESTE ES EL SHAPE:
(58110, 14)
INFO DE DF


<bound method DataFrame.info of              game_date_est  game_sequence   game_id  game_status_id  \
0      1946-11-01 00:00:00            NaN  24600001               3   
1      1946-11-02 00:00:00            NaN  24600003               3   
2      1946-11-02 00:00:00            NaN  24600002               3   
3      1946-11-02 00:00:00            NaN  24600004               3   
4      1946-11-02 00:00:00            NaN  24600005               3   
...                    ...            ...       ...             ...   
58105  2023-06-07 00:00:00            1.0  42200403               3   
58106  2023-06-09 00:00:00            1.0  42200404               3   
58107  2023-06-12 00:00:00            1.0  42200405               3   
58108  2023-02-19 00:00:00            1.0  32200001               3   
58109  2023-02-19 00:00:00            1.0  32200001               3   

      game_status_text         gamecode  home_team_id  visitor_team_id  \
0                  NaN  19461101/NYKHUS  

In [11]:
#CORREGIR FECHAS
df['game_date_est'] = pd.to_datetime(df['game_date_est'])
#REVISAR NULOS
df.isnull().sum()

game_date_est                           0
game_sequence                       25532
game_id                                 0
game_status_id                          0
game_status_text                    25986
gamecode                                0
home_team_id                            0
visitor_team_id                         0
season                                  0
live_period                             0
live_pc_time                        56086
natl_tv_broadcaster_abbreviation    51907
live_period_time_bcast                  0
wh_status                               0
dtype: int64

In [12]:
#Hay 14 columnas
#live_pc_time con 2024 filas llenas de 58110 (sólo un 3.5%) // Por lo que la columna es prescindible
#natl_tv_broadcaster_abbreviation con 6203 filas llenas de 58110 (sólo un 10%)


#¿Qué columnas están divididas?
df['game_date_est'] = pd.to_datetime(df['game_date_est'])

# 1. Creamos una columna temporal para el año
df['year'] = df['game_date_est'].dt.year

# 2. Agrupamos por año y contamos los nulos de una columna específica
nulos_por_anio = df[df['game_status_text'].isnull()].groupby('year').size()

# 3. Mostramos los años con más nulos
print("Años con mayor cantidad de nulos en game_status_text:")
print(nulos_por_anio.sort_values(ascending=False).head(10))

Años con mayor cantidad de nulos en game_status_text:
year
1991    1065
1993    1056
1990    1055
1992     991
1994     990
1989     984
1988     937
1984     906
1986     877
1987     868
dtype: int64


In [13]:
# Agrupamos, contamos y ordenamos por el índice (que es el año)
nulos_todos_anios = df[df['game_status_text'].isnull()].groupby('year').size().sort_index()

# Para ver la lista completa
with pd.option_context('display.max_rows', None):
    print(nulos_todos_anios)

nulos_todos_anios.to_csv('reporte_nulos_por_anio.csv')

year
1946     103
1947     237
1948     244
1949     404
1950     426
1951     319
1952     317
1953     303
1954     277
1955     264
1956     281
1957     255
1958     250
1959     250
1960     157
1962     159
1963     322
1964     333
1965     304
1966     190
1967     228
1968     491
1969     493
1970     290
1971     308
1972     630
1973     645
1974     600
1975     410
1976      46
1977     364
1978     777
1979     760
1980     830
1981     733
1982     854
1983     855
1984     906
1985     833
1986     877
1987     868
1988     937
1989     984
1990    1055
1991    1065
1992     991
1993    1056
1994     990
1995     715
dtype: int64


In [14]:
#¿Cuantos partidos hay dentro del grupo de estudio? ¿Son suficientes?
# Aseguramos que la columna sea datetime 

# Creamos el filtro para fechas mayores o iguales al 1 de enero de 2004
datos_desde_2004 = df[df['game_date_est'] >= '2004-01-01']

# Contamos las filas
cantidad = len(datos_desde_2004)

print(f"Hay {cantidad} partidos registrados desde el 2004 hasta el 2023.")

Hay 23100 partidos registrados desde el 2004 hasta el 2023.


In [15]:
#Eliminando filas que no sirven para nuestro estudio
#Sobrescribimos el df manteniendo solo del 2004 hacia adelante
df = df[df['game_date_est'] >= '2004-01-01']

#Verificamos el resultado
print(f"Nuevo total de filas: {len(df)}")
print(f"Fecha mínima actual: {df['game_date_est'].min()}")

Nuevo total de filas: 23100
Fecha mínima actual: 2004-01-02 00:00:00


In [16]:
# Eliminamos la columna live_pc_time ya que tiene 97% de nulos
# Usamos 'inplace=True' para que el cambio se guarde directamente en el df
df.drop(columns=['live_pc_time'], inplace=True)

# Verificamos que ya no esté
print(df.columns)

Index(['game_date_est', 'game_sequence', 'game_id', 'game_status_id',
       'game_status_text', 'gamecode', 'home_team_id', 'visitor_team_id',
       'season', 'live_period', 'natl_tv_broadcaster_abbreviation',
       'live_period_time_bcast', 'wh_status', 'year'],
      dtype='object')


C:\Users\palom\AppData\Local\Temp\ipykernel_24944\2558554968.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['live_pc_time'], inplace=True)


In [17]:
#Eliminamos la columna de canales de television que han transmitido los partidos, ya que tiene 90% de nulos
df.drop(columns=['natl_tv_broadcaster_abbreviation'], inplace=True)

C:\Users\palom\AppData\Local\Temp\ipykernel_24944\309770700.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['natl_tv_broadcaster_abbreviation'], inplace=True)


In [18]:
#REVISAR NULOS
df.isnull().sum()

game_date_est             0
game_sequence             0
game_id                   0
game_status_id            0
game_status_text          0
gamecode                  0
home_team_id              0
visitor_team_id           0
season                    0
live_period               0
live_period_time_bcast    0
wh_status                 0
year                      0
dtype: int64

In [20]:
#AL LIMPIAR LINE_SCORE NOS DIMOS CUENTA DE QUE LA CANTIDAD DE REGISTROS NO COINCIDE CON LA DE ESTE CSV.
print(df.shape)
#CORROBORAMOS DUPLICADOS:
print(f"Partidos únicos en game_summary: {df['game_id'].nunique()}")

(23100, 13)
Partidos únicos en game_summary: 23040


In [23]:
df.drop_duplicates(subset=['game_id'], keep='first', inplace=True)
print(df.shape)
print(f"Partidos únicos en game_summary: {df['game_id'].nunique()}")

(23040, 13)
Partidos únicos en game_summary: 23040


C:\Users\palom\AppData\Local\Temp\ipykernel_24944\2927515107.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(subset=['game_id'], keep='first', inplace=True)


In [ ]:
df2= pd.read_csv('csv/line_score.csv')
ids_comunes = set(df2['game_id']).intersection(set(df['game_id']))
df = df[df['game_id'].isin(ids_comunes)]
print(f"Partidos en Game Summary (df2): {len(df)}")
df.to_csv('csv/cleaned/game_summary_cleaned.csv', index=False)

Partidos en Game Summary (df2): 23032
